In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from collections import Counter

from efficientdet.data_pipeline.utils import xywh_to_xyxy
from efficientdet.data_pipeline.anchors import AnchorBox
from efficientdet.data_pipeline.target_encoder import TargetEncoder
from efficientdet.data_pipeline.input_dataset import create_combined_dataset, create_images_dataset, create_bbox_dataset, create_labels_dataset, name_targets
from efficientdet.constants import IMG_SHAPE

import imgaug.augmenters as aug_lib
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

# Load data

In [ ]:
path_train = '../data/train.csv'

df_train = pd.read_csv(path_train, dtype={'img_path': str, 'x1': 'int32', 'y1': 'int32', 'x2': 'int32', 'y2': 'int32', 'label': 'int32'})
df_train.head(2)

# Create tf dataset

In [ ]:
target_encoder = TargetEncoder()

ds_norm = create_combined_dataset(path_train, batch_size=1, augment=False)
ds_aug = create_combined_dataset(path_train, batch_size=1, augment=True)

# Visualize augmentations

In [ ]:
def add_box(ax, bbox, ec='r', fc='none', lw=0.5):
    x1, y1, x2, y2 = bbox
    height = y2 - y1
    width = x2 - x1
    rect = Rectangle((x1, y1), width, height, ec=ec, fc=fc, lw=lw)
    ax.add_patch(rect)


def plot_anchors(ax, x):
    img, box_targets, cls_targets = x[0][0].numpy(), x[1]['regression'][0].numpy(), x[1]['classification'][0].numpy()
    valid_anchors = all_anchors[box_targets[:,-1] == 1]
    cls_target_counter = Counter(cls_targets)
    positive_anchor_keys = [k for k in cls_target_counter.keys() if k >= 0]
    n_positive_anchors = "(label: anchors) " + ", ".join(["{}: {}".format(int(k), cls_target_counter[k]) for k in positive_anchor_keys])
    ax.set_title(n_positive_anchors, fontsize=14)
    ax.imshow(img)
    # plot all anchors
    for bbox in xywh_to_xyxy(valid_anchors):
        add_box(ax, bbox)

In [ ]:
n_rows = 10
fig, axes = plt.subplots(n_rows, 2, figsize=(10, n_rows*5))

all_anchors = target_encoder._anchor_box.get_all_anchors(512, 512).numpy()
print("Left: input data without augmentation. Right: same input data with augmentation")
print("Cyan bounding box is ground truth (only available in the left plots)")
for i, (x_norm, x_aug, (_, g)) in enumerate(zip(ds_norm.take(n_rows), ds_aug.take(n_rows), df_train.groupby('img_path'))):
    ax1, ax2 = axes[i, :]
    plot_anchors(ax1, x_norm)
    plot_anchors(ax2, x_aug)
    # plot all ground truth boxes
    for _, row in g.iterrows():
        add_box(ax1, row[['x1', 'y1', 'x2', 'y2']], ec='cyan', lw=1)